In [1]:
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

#remove comment on first execution
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ketan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ketan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ketan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
def pattern_replacement(pattern, tweet, replacement):
    r = re.findall('"' + pattern + '[\w]*"', tweet)
    for i in r:
        tweet = re.sub(i, replacement, tweet)
    return tweet;


In [8]:
def preprocess_data(tweet):
    tweet = pattern_replacement('@', tweet, '')

    #.. Removing
    tweet = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags=re.MULTILINE)

    #...Removing non-alphabets from the tweet
    r = re.findall("[^A-Za-z]", tweet)
    for i in r:
        tweet = tweet.replace(i, " ")

    
    #...Removing preposiitons, conjuctions and pronouns from the tweet
    words = nltk.word_tokenize(tweet)
    tags = nltk.pos_tag(words)
    '''
    PRP	Personal pronoun
    DT	Determiner
    CC	Coordinating conjunction
    IN	Preposition or subordinating conjunction
    PRP$	Possessive pronoun
    VBP	Verb, non-3rd person singular present
    NNP	Proper noun, singular
    VBZ	Verb, 3rd person singular present
    VB	Verb, base form
    MD	Modal
    RB	Adverb
    VBD	Verb, past tense
    WP	Wh-pronoun
    CD	Cardinal number
    WRB	Wh-adverb
    WDT	Wh-determiner
    '''    
    
    #del_tags = ['PRP','DT','CC', 'IN', 'PRP$', 'VBP', 'NNP', 'VBZ', 'VB', 'MD', 'RB', 'VBD', 'WP', 'CD', 'WRB', 'WDT']
    del_tags = ['PRP','DT','CC', 'IN', 'PRP$', 'VBP', 'MD', 'WP', 'CD', 'WRB', 'WDT']

    new_tags = []
    for ord_pair in tags:
        if ord_pair[1] not in del_tags and len(ord_pair[0]) > 3:
            new_tags.append(ord_pair[0])

    #...Removing Stopwords from the tweet
    new_tags = [w for w in new_tags if not w in stop_words]

    tweet = " ".join(new_tags)
    
    return tweet


In [9]:
df = pd.read_csv("datasets/socialmedia-disaster-tweets.csv",encoding = "ISO-8859-1")

#print(df)

df2=pd.DataFrame(columns=['TweetId', 'Tweet', 'Label'])
#print(df2)
len_df = len(df)
#print(len_df)
#tweets=df['Tweet']
#print(tweets)

TweetIds=[]
Tweets=[]
Labels=[]

for index in range(len_df):
    tweet = df.iloc[index]['Tweet']
    
   #...Removing @Users from the tweet
    tweet=preprocess_data(tweet)
    
    #...Replacing the tweet in the DataFrame
    Tweets.append(tweet)
    TweetIds.append(df['ID'][index])
    tweets_labeled=df['Label'][index]
    if tweets_labeled=='Related and informative' or tweets_labeled=='Relevant':
      Labels.append(1)
    else:
      Labels.append(0)
    #df2.append({'TweetId': TweetId,'Tweet':tweet,'label':label},ignore_index=True)
    #df2['TweetId'][index] = df['TweetID'][index]
    #df['Tweet'][index] = tweet
    #print(df2)
    

In [10]:
df2['TweetId']=TweetIds
df2['Tweet']=Tweets
df2['Label']=Labels

print(df2)

TweetId                                              Tweet  Label
0      778243823                       Just happened terrible crash      1
1      778243824              Deeds Reason earthquake ALLAH Forgive      1
2      778243825    Heard earthquake different cities safe everyone      1
3      778243826    forest fire spot pond geese fleeing street save      1
4      778243827                      Forest fire Ronge Sask Canada      1
...          ...                                                ...    ...
10871  778261105                                     Volcano Hawaii      1
10872  778261106  Police investigating bike collided Little Port...      1
10873  778261107  Latest More Homes Razed Northern California Wi...      1
10874  778261108                   issues Hazardous Weather Outlook      1
10875  778261109  CityofCalgary activated Municipal Emergency Pl...      1

[10876 rows x 3 columns]


In [11]:
df2.to_csv('processedData/Cleaned_Data.csv')